<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/examples/model_assisted_labeling/image_mal.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/develop/examples/model_assisted_labeling/image_mal.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Image Annotation Import
* This notebook will provide examples of each supported annotation type for image assets. It will cover the following:
    * Model-assisted labeling - used to provide pre-annotated data for your labelers. This will enable a reduction in the total amount of time to properly label your assets. Model-assisted labeling does not submit the labels automatically, and will need to be reviewed by a labeler for submission.
    * Label Import - used to provide ground truth labels. These can in turn be used and compared against prediction labels, or used as benchmarks to see how your labelers are doing.

* For information on what types of annotations are supported per data type, refer to this [documentation](https://docs.labelbox.com/docs/model-assisted-labeling#option-1-import-via-python-annotation-types-recommended)

* Notes:
    * If you are importing more than 1,000 mask annotations at a time, consider submitting separate jobs, as they can take longer than other annotation types to import.
    * Wait until the import job is complete before opening the Editor to make sure all annotations are imported properly.

# Installs

In [1]:
!pip install -q 'labelbox[data]'

# Imports

In [2]:
from labelbox.schema.ontology import OntologyBuilder, Tool, Classification, Option
from labelbox import Client, LabelingFrontend, LabelImport, MALPredictionImport
from labelbox.data.annotation_types import (
    Label, ImageData, ObjectAnnotation, MaskData,
    Rectangle, Point, Line, Mask, Polygon,
    Radio, Checklist, Text,
    ClassificationAnnotation, ClassificationAnswer
)
from labelbox.data.serialization import NDJsonConverter
import uuid
import json
import numpy as np

# API Key and Client
Provide a valid api key below in order to properly connect to the Labelbox Client.

In [3]:
# Add your api key
API_KEY = None
client = Client(api_key=API_KEY)

INFO:labelbox.client:Initializing Labelbox client at 'https://api.labelbox.com/graphql'


---- 
### Steps
1. Make sure project is setup
2. Collect annotations
3. Upload

### Project setup

We will be creating two projects, one for model-assisted labeling, and one for label imports

In [4]:
ontology_builder = OntologyBuilder(
    tools=[
        Tool(tool=Tool.Type.BBOX, name="box"),
        Tool(tool=Tool.Type.LINE, name="line"),
        Tool(tool=Tool.Type.POINT, name="point"),
        Tool(tool=Tool.Type.POLYGON, name="polygon"),    
        Tool(tool=Tool.Type.SEGMENTATION, name="mask")],
    classifications=[
        Classification(class_type=Classification.Type.TEXT, instructions="text"),
        Classification(class_type=Classification.Type.CHECKLIST, instructions="checklist", options=[
            Option(value="first_checklist_answer"),
            Option(value="second_checklist_answer")            
        ]),
        Classification(class_type=Classification.Type.RADIO, instructions="radio", options=[
            Option(value="first_radio_answer"),
            Option(value="second_radio_answer")
        ]),                                      
])


In [5]:
mal_project = client.create_project(name="image_mal_project")
li_project = client.create_project(name="image_label_import_project")


dataset = client.create_dataset(name="annotation_import_demo_dataset")
test_img_url = "https://raw.githubusercontent.com/Labelbox/labelbox-python/develop/examples/assets/2560px-Kitano_Street_Kobe01s5s4110.jpg"
data_row = dataset.create_data_row(row_data=test_img_url)
editor = next(client.get_labeling_frontends(where=LabelingFrontend.name == "Editor"))

mal_project.setup(editor, ontology_builder.asdict())
mal_project.datasets.connect(dataset)

li_project.setup(editor, ontology_builder.asdict())
li_project.datasets.connect(dataset)

### Create Label using Annotation Type Objects
* It is recommended to use the Python SDK's annotation types for importing into Labelbox.

### Object Annotations

In [6]:
point = Point(x=100,y=100)
point_annotation = ObjectAnnotation(value=point, name="point")

rectangle = Rectangle(start=Point(x=30,y=30), end=Point(x=200,y=200))
rectangle_annotation = ObjectAnnotation(value=rectangle, name="box")

line = Line(points=[Point(x=60,y=70), Point(x=65,y=100), Point(x=80,y=130), Point(x=40,y=200)])
line_annotation = ObjectAnnotation(value=line, name="line")

polygon = Polygon(points=[Point(x=100,y=100), Point(x=110,y=110), Point(x=130,y=130), Point(x=170,y=170), Point(x=220,y=220)])
polygon_annotation = ObjectAnnotation(value=polygon, name="polygon")

array = np.zeros([128, 128, 3], dtype=np.uint8)
mask_data = MaskData(arr=array)
mask = Mask(mask=mask_data, color=(0, 0, 0))
mask_annotation = ObjectAnnotation(value=mask, name="mask")

### Classification Annotations

In [7]:
text = Text(answer="the answer to the text question")
text_annotation = ClassificationAnnotation(value=text, name="text")

checklist = Checklist(answer=[ClassificationAnswer(name="first_checklist_answer"),ClassificationAnswer(name="second_checklist_answer")])
checklist_annotation = ClassificationAnnotation(value=checklist, name="checklist")

radio = Radio(answer = ClassificationAnswer(name = "second_radio_answer"))
radio_annotation = ClassificationAnnotation(value=radio, name="radio")

### Create a Label object with all of our annotations

In [8]:
image_data = ImageData(uid=data_row.uid)

label = Label(
    data=image_data,
    annotations = [
        point_annotation, rectangle_annotation, line_annotation, polygon_annotation, mask_annotation,
        text_annotation, checklist_annotation, radio_annotation
    ]
)

# Create urls to mask data for upload
def signing_function(obj_bytes: bytes) -> str:
    url = client.upload_data(content=obj_bytes, sign=True)
    return url


label.add_url_to_masks(signing_function)

label.__dict__

{'uid': None,
 'data': ImageData(im_bytes=None,file_path=None,url=None,arr=None),
 'annotations': [ObjectAnnotation(name='point', feature_schema_id=None, extra={}, value=Point(extra={}, x=100.0, y=100.0), classifications=[]),
  ObjectAnnotation(name='box', feature_schema_id=None, extra={}, value=Rectangle(extra={}, start=Point(extra={}, x=30.0, y=30.0), end=Point(extra={}, x=200.0, y=200.0)), classifications=[]),
  ObjectAnnotation(name='line', feature_schema_id=None, extra={}, value=Line(extra={}, points=[Point(extra={}, x=60.0, y=70.0), Point(extra={}, x=65.0, y=100.0), Point(extra={}, x=80.0, y=130.0), Point(extra={}, x=40.0, y=200.0)]), classifications=[]),
  ObjectAnnotation(name='polygon', feature_schema_id=None, extra={}, value=Polygon(extra={}, points=[Point(extra={}, x=100.0, y=100.0), Point(extra={}, x=110.0, y=110.0), Point(extra={}, x=130.0, y=130.0), Point(extra={}, x=170.0, y=170.0), Point(extra={}, x=220.0, y=220.0), Point(extra={}, x=100.0, y=100.0)]), classifications=[

### Model Assisted Labeling 

To do model-assisted labeling, we need to convert a Label object into an NDJSON. 

This is easily done with using the NDJSONConverter class

We will create a Label called mal_label which has the same original structure as the label above

Notes:
* Each label requires a valid feature schema id. We will assign it using our built in `assign_feature_schema_ids` method
* the NDJsonConverter takes in a list of labels

In [9]:
mal_label = Label(
    data=image_data,
    annotations = [
        point_annotation, rectangle_annotation, line_annotation, polygon_annotation, mask_annotation,
        text_annotation, checklist_annotation, radio_annotation
    ]
)

label.add_url_to_masks(signing_function)

mal_label.assign_feature_schema_ids(ontology_builder.from_project(mal_project))

ndjson_labels = list(NDJsonConverter.serialize([mal_label]))

ndjson_labels

[{'uuid': '4660b7b9-c27e-4bd6-b0ed-0cc43bb15097',
  'dataRow': {'id': 'ckzso867f7lof0zc3d7c222gq'},
  'schemaId': 'ckzso86t97lou0zc3b0ywg7qu',
  'classifications': [],
  'point': {'x': 100.0, 'y': 100.0}},
 {'uuid': '0d6df02b-3780-450b-a816-718f53830e12',
  'dataRow': {'id': 'ckzso867f7lof0zc3d7c222gq'},
  'schemaId': 'ckzso86t97loq0zc3chj70unt',
  'classifications': [],
  'bbox': {'top': 30.0, 'left': 30.0, 'height': 170.0, 'width': 170.0}},
 {'uuid': 'afe350ba-8173-46bd-b1cf-a7adbccadcd5',
  'dataRow': {'id': 'ckzso867f7lof0zc3d7c222gq'},
  'schemaId': 'ckzso86t97los0zc3cgbh1zkk',
  'classifications': [],
  'line': [{'x': 60.0, 'y': 70.0},
   {'x': 65.0, 'y': 100.0},
   {'x': 80.0, 'y': 130.0},
   {'x': 40.0, 'y': 200.0}]},
 {'uuid': '5d1ef8b1-533c-4f28-9adc-b6554a0828cb',
  'dataRow': {'id': 'ckzso867f7lof0zc3d7c222gq'},
  'schemaId': 'ckzso86t97low0zc32hk86suf',
  'classifications': [],
  'polygon': [{'x': 100.0, 'y': 100.0},
   {'x': 110.0, 'y': 110.0},
   {'x': 130.0, 'y': 130.0}

In [10]:
upload_job = MALPredictionImport.create_from_objects(
    client = client, 
    project_id = mal_project.uid, 
    name="upload_label_import_job", 
    predictions=ndjson_labels)

In [11]:
# Errors will appear for each annotation that failed.
# Empty list means that there were no errors
# This will provide information only after the upload_job is complete, so we do not need to worry about having to rerun
print("Errors:", upload_job.errors)

INFO:labelbox.schema.annotation_import:Sleeping for 10 seconds...


Errors: []


### Label Import

Label import is very similar to model-assisted labeling. We will need to re-assign the feature schema before continuing, 
but we can continue to use our NDJSonConverter

We will create a Label called li_label which has the same original structure as the label above

In [12]:
#for the purpose of this notebook, we will need to reset the schema ids of our checklist and radio answers
checklist = Checklist(answer=[ClassificationAnswer(name="first_checklist_answer"),ClassificationAnswer(name="second_checklist_answer")])
checklist_annotation = ClassificationAnnotation(value=checklist, name="checklist")
radio = Radio(answer = ClassificationAnswer(name = "second_radio_answer"))
radio_annotation = ClassificationAnnotation(value=radio, name="radio")

li_label = Label(
    data=image_data,
    annotations = [
        point_annotation, rectangle_annotation, line_annotation, polygon_annotation, mask_annotation,
        text_annotation, checklist_annotation, radio_annotation
    ]
)

li_label.assign_feature_schema_ids(ontology_builder.from_project(li_project))

ndjson_labels = list(NDJsonConverter.serialize([li_label]))

ndjson_labels

[{'uuid': '642e4455-a3a7-4fdc-8155-cedf19464149',
  'dataRow': {'id': 'ckzso867f7lof0zc3d7c222gq'},
  'schemaId': 'ckzso8a3d7iro0zbsha374jm8',
  'classifications': [],
  'point': {'x': 100.0, 'y': 100.0}},
 {'uuid': 'a492c8d9-a7a1-4a88-a879-a261ae4e99b4',
  'dataRow': {'id': 'ckzso867f7lof0zc3d7c222gq'},
  'schemaId': 'ckzso8a3d7irk0zbs09fe9eyy',
  'classifications': [],
  'bbox': {'top': 30.0, 'left': 30.0, 'height': 170.0, 'width': 170.0}},
 {'uuid': '6d35b326-152d-42c2-ac6a-deb54e7763ac',
  'dataRow': {'id': 'ckzso867f7lof0zc3d7c222gq'},
  'schemaId': 'ckzso8a3d7irm0zbsfvoybop9',
  'classifications': [],
  'line': [{'x': 60.0, 'y': 70.0},
   {'x': 65.0, 'y': 100.0},
   {'x': 80.0, 'y': 130.0},
   {'x': 40.0, 'y': 200.0}]},
 {'uuid': '0b81b854-5362-4a16-97fc-8cd890528e89',
  'dataRow': {'id': 'ckzso867f7lof0zc3d7c222gq'},
  'schemaId': 'ckzso8a3d7irq0zbshcv48vkx',
  'classifications': [],
  'polygon': [{'x': 100.0, 'y': 100.0},
   {'x': 110.0, 'y': 110.0},
   {'x': 130.0, 'y': 130.0}

In [13]:
upload_job = LabelImport.create_from_objects(
    client = client, 
    project_id = li_project.uid, 
    name="upload_label_import_job", 
    labels=ndjson_labels)

In [14]:
print("Errors:", upload_job.errors)

INFO:labelbox.schema.annotation_import:Sleeping for 10 seconds...


Errors: []
